# Chapter 3: Gradient-based MCMC: Langevin and Hamiltonian methods

**note: chapter under construction**

## Table of Contents:
* [Introduction](#Introduction)
* [Unadjusted Langevin algorithm](#ULA)
* [Metropolis-adjusted Langevin algorithm](#MALA)
* [Hamiltonian Monte Carlo](#HMC)
* [No-U-Turn Sampler](#NUTS)


## Introduction <a class="anchor" id="introduction"></a>


## Unadjusted Langevin algorithm <a class="anchor" id="ULA"></a>

* In the Unadjusted Langevin algorithm (ULA), new states are proposed using (overdamped) Langevin dynamics.


## Metropolis-adjusted Langevin algorithm <a class="anchor" id="MALA"></a>

* Let $p$ denote a probability density on $\mathbb{R}^d$, from which it is desired to draw an ensemble of iid samples. We consider the overdamped Langevin Itô diffusion
 
$$
\begin{equation*}
\dfrac{\mathrm{d}\boldsymbol{x}_t}{\mathrm{d} t} = \dfrac{1}{2}\nabla\log\pi(\boldsymbol{x}) + \dfrac{\mathrm{d} W_t}{\mathrm{d} t},
\end{equation*}
$$

driven by the time derivative of a standard Brownian motion $W$.

* In the limit, as $t\to\infty$, the probability distribution of $X(t)$ approaches a stationary distribution, which is also invariant under the diffusion. It turns out that this distribution is $p$.
 
* Approximate sample paths of the Langevin diffusion can be generated by the Euler--Maruyama method with a fixed time step $\varepsilon>0$. We set $\boldsymbol{x}_{0}$ and then recursively define an approximation to the true solution by
 
$$
\begin{equation*}
\boldsymbol{x}_{k+1} := \boldsymbol{x}_{k} + \dfrac{\varepsilon^2}{2} \nabla \log p(\boldsymbol{x}_{k})+ \varepsilon\,\boldsymbol{\xi}_{k},
\end{equation*}
$$

where each $\boldsymbol{\xi}_k\sim\mathcal{N}(\boldsymbol{0},\boldsymbol{I}_d)$.

* In contrast to the Euler--Maruyama method for simulating the Langevin diffusion. MALA incorporates an additional step. We consider the above update rule as defining a proposal $\tilde{\boldsymbol{x}}$ for a new state:
 
$$
\begin{equation}
{\tilde{\boldsymbol{x}}}_{k+1}:=\boldsymbol{x}_{k}+\dfrac{\varepsilon^2}{2} \nabla \log p(\boldsymbol{x}_{k})+ \varepsilon\,\boldsymbol{\xi}_{k};
\end{equation}
$$

this proposal is accepted or rejected according to the MH algorithm.

* That is, the acceptance probability is:
 
 $$
\begin{equation}
\alpha(\boldsymbol{x}_k,{\tilde{\boldsymbol{x}}}_{k+1}) = \min\left(1,\dfrac{p({\tilde{\boldsymbol{x}}}_{k+1})q(\boldsymbol{x}_k \,\vert\, {\tilde{\boldsymbol{x}}}_{k+1})} {p(\boldsymbol{x}_k)q({\tilde{\boldsymbol{x}}}_{k+1} \,\vert\, \boldsymbol{x}_k)}\right);
\end{equation}
$$

where the proposal has the form

$$
\begin{equation}
q(\boldsymbol{x}' \,\vert\, \boldsymbol{x}) = \mathcal{N}\left(\boldsymbol{x}';\, \boldsymbol{x}+\dfrac{\varepsilon^2}{2} \nabla \log p(\boldsymbol{x}),\varepsilon^2\boldsymbol{I}_d\right).
\end{equation}
$$

* The combined dynamics of the Langevin diffusion and the MH algorithm satisfy the detailed balance conditions necessary for the existence of a unique, invariant, stationary distribution.

* For limited classes of target distributions, the optimal acceptance rate for this algorithm can be shown to be $0.574$; this can be used to tune $\varepsilon$.
 

## Hamiltonian Monte Carlo <a class="anchor" id="HMC"></a>

* Hamiltonian Monte Carlo (HMC): use Hamiltonian dynamics to simulate particle trajectories.
 
* Define a Hamiltonian function in terms of the target distribution.
 
* Introduce an auxiliary momentum variables, which typically have independent Gaussian distributions.
 
* Hamiltonian dynamics operate on a $d$-dimensional position vector $\boldsymbol{x}$, and a $d$-dimensional momentum vector $\boldsymbol{r}$, so that the full state space has $2d$ dimensions. The system is described by a function of $\boldsymbol{x}$ and $\boldsymbol{r}$ known as the Hamiltonian $H(\boldsymbol{x}, \boldsymbol{r})$.

* In HMC, one uses Hamiltonian functions that can be written as (closed-system dynamics):
 
$$
\begin{equation}
H(\boldsymbol{x}, \boldsymbol{r}) = \underbrace{U(\boldsymbol{x})}_{\text{potential energy}} + \underbrace{K(\boldsymbol{r},\boldsymbol{x})}_{\text{kinetic energy}}.
\end{equation}
$$

* The potential energy is completely determined by the target distribution, indeed $U(\boldsymbol{x})$ is equal to the logarithm of the target distribution $p$.

* The kinetic energy is unconstrained and must be specified by the implementation.
 
* The Hamiltonian is an energy function for the joint state of position-momentum, and so defines a joint distribution for them as follows:
 
$$
\begin{equation}
p(\boldsymbol{x}, \boldsymbol{r}) = \dfrac{1}{Z}\exp\left(-\dfrac{H(\boldsymbol{x}, \boldsymbol{r})}{T}\right) = \dfrac{1}{Z}\exp\left(-{U(\boldsymbol{x})}\right)\exp\left(-{K(\boldsymbol{r})}\right).
\end{equation}
$$

* There are several ways to set the kinetic energy (density of the auxiliary momentum):
  - Euclidean--Gaussian kinetic energy: using a fixed covariance $\boldsymbol{M}$ estimated from the position parameters, $K(\boldsymbol{p},\boldsymbol{q}) = \dfrac{1}{2} \boldsymbol{r}^T\boldsymbol{M}^{-1}\boldsymbol{r} + \log(\abs{\boldsymbol{M}}) + \text{const}$.
  - Riemann--Gaussian kinetic energy: unlike the Euclidean metric, varies as one moves through parameter space, $K(\boldsymbol{r}, \boldsymbol{x}) = \dfrac{1}{2} \boldsymbol{r}^T\boldsymbol{\Sigma}(\boldsymbol{x})^{-1}\boldsymbol{r} + \dfrac{1}{2}\log(\abs{\boldsymbol{\Sigma}(\boldsymbol{x})}) + \text{const}$.
  - Non-Gaussian kinetic energies.
  
* Hamilton's equations read as follows:

$$
\begin{align}
\dfrac{\mathrm{d} \boldsymbol{x}}{\dd t} &= +\frac{\partial H}{\partial \boldsymbol{r}} = \boldsymbol{M}^{-1}\boldsymbol{r} \\
\dfrac{\mathrm{d} \boldsymbol{r}}{\mathrm{d} t} &= -\dfrac{\partial H}{\partial \boldsymbol{x}} = -\dfrac{\partial K}{\partial \boldsymbol{x}} - \dfrac{\partial U}{\partial \boldsymbol{x}},
\end{align}
$$

where $\dfrac{\partial U}{\partial \boldsymbol{x}}$ is the gradient of the logarithm of the target density.

* Discretizing Hamilton's equations:
  - Euler's method (no!).
  - Modified Euler's method (a bit better!).
  - Symplectic integrators: the leapfrog method (the standard choice!).
  
* Hamiltonian dynamics are time-reversible and volume-preserving.

* The dynamics keep the Hamiltonian invariant. A Hamiltonian trajectory will (if simulated exactly) move within a hypersurface of constant probability density.

* Each iteration of the HMC algorithm has two steps. Both steps leave the joint distribution of $p(\boldsymbol{x}, \boldsymbol{r})$ invariant (detailed balance).
  - In the first step, new values of $\boldsymbol{r}$ are randomly drawn from their Gaussian distribution, independently of the current $\boldsymbol{x}$.
  - In the second step, a Metropolis update is performed, using Hamiltonian dynamics to propose a new state.
  - Optimal acceptance rate is 0.65. The step size $\varepsilon$ and trajectory length $L$ need to be tuned.
  

## No-U-Turn sampler <a class="anchor" id="NUTS"></a>

* HMC is an algorithm that avoids the random walk behavior and sensitivity to correlated parameters that plague many MCMC methods by taking a series of steps informed by first-order gradient information.

* However, HMC's performance is highly sensitive to two user-specified parameters: a step size $\varepsilon$ and a desired number of steps $L$.

*  The No-U-Turn Sampler (NUTS), an extension to HMC that eliminates the need to set a number of steps $L$, as well as the step-size.

* We simulate in discrete time steps, and to make sure you explore the parameter space properly you simulate steps in one direction and the twice as many in the other direction, turn around again, etc. At some point you want to stop this and a good way of doing that is when you have done a U-turn (i.e., appear to have gone all over the place).

* NUTS begins by introducing an auxiliary variable with conditional distribution. After re-sampling from this distribution, NUTS uses the leapfrog integrator to trace out a path forwards and backwards in fictitious time. First running forwards or backwards 1 step, then forwards or backwards 2 steps, then forwards or backwards 4 steps, etc.

* This doubling process implicitly builds a balanced binary tree whose leaf nodes correspond to position-momentum state. The doubling is stopped when the subtrajectory from the leftmost to the rightmost nodes of any balanced subtree of the overall binary tree starts to double back on itself (i.e., the fictional particle starts to make a U-turn).

* At this point NUTS stops the simulation and samples from among the set of points computed during the simulation, taking care to preserve detailed balance.

* To adapt the step-size, NUTS uses a modified dual averaging algorithm during the burn-in phase.

* The good thing about NUTS is that proposals are made based on the shape of the posterior and they can happend at the other end of the distribution. In contrast, MH makes proposals within a ball, and Gibbs sampling only moves along one (or at least very few) dimensions at a time.